# Integração numérica

Também conhecido na literatura como quadratura.

Como calcular

$$
\int_a^b f(\xi)\:d\xi
$$

para uma função qualquer?

 * Na mão: difícil mesmo nos casos mais simples
 * No computador com cálculo simbólico: útil algumas vezes mas pouco eficiente
 * Métodos numéricos - na prático o que é usado.


# Usar as funções aproximadas!

É bom lembrar que chegamos em várias maneiras para aproximar uma função:

$$
y = f(x) \approx y^\delta = \sum_{i=1}^n a_i\phi_i(x)
$$

Uma primeira possibilidade é integrar termo a termo:

$$
\int_a^bf(\xi)\:d\xi \approx \sum_{i=1}^n a_i\int_a^b\phi_i(\xi)\:d\xi
$$

Como $\phi_i(x)$ foram escolhidos levando em conta simplicidade, esta abordagem pode ser utilizada. 

# Regra do ponto médio:

Dados os pontos $x_1, x_2, \ldots, x_n$, usando interpoladores constantes

In [ ]:
using PyPlot
using CurveFit
using BenchmarkTools

In [ ]:
x = 0:0.1:1
xx = 0:0.002:1
f(x) = sin(π*x) + 0.5 + cos(x)
y = f.(x);
yy = f.(xx);

In [ ]:
plot(xx, yy, "b-")
plot(x, y, "rs")

nx = length(x)

lty = "k-"

for i in 1:nx-1
    xm = 0.5 * (x[i] + x[i+1])
    ym = f(xm)
    plot([x[i], x[i], x[i+1], x[i+1]], [0, ym, ym, 0], lty )
    plot([xm, xm], [0, ym], "k:")
    
end


Analisando a figura acima, pode-se facilmante ver que a integral vale (para pontos igualmente espaçados):

$$
\int_a^b f(\xi)\:d\xi \approx \frac{b-a}{n-1} \sum_{i=1}^{n-1}\left[ f\left(x_{i+\frac{1}{2}}\right) \right]
$$

In [ ]:

function midrule(a, b, f, n)
    dx = (b-a) / n
    x = range(a, b, length=n+1)
    s = 0.0
    for i in 1:n
        s += f(x[i]+dx/2)
    end
    
    return dx * s
end
    

In [ ]:
## Análise de convergência:

nn = 2:1000
a = 0.0
b = 1.0
Ie = 2/π + 0.5 + sin(1)

Inn = midrule.(a, b, f, nn);
err = abs.(Inn .- Ie);

In [ ]:
power_fit(nn, err)

In [ ]:
loglog(nn, err, "bo")
xlabel("n")
ylabel("ε")
nn1 = 10:300
ee1 = 5 .* nn1.^(-2)
plot(nn1, ee1, "r--")
text(50, 0.005, L"$\mathcal{O}(n^{-2})$")

In [ ]:
@benchmark midrule(a, b, f, 100000)

Assim, a estimativa do erro vale:

$$
\varepsilon = \mathcal{O}\left(n^{-2}\right)
$$

# Regra do trapézio

Agora, ao invés de usar interpolação consante entre os pontos, pode-se ligar os pontos por retas.

In [ ]:
function trapezoidal(a, b, f, n)
    x = range(a, b, length=n+1)
    
    s = f(x[1])/2 + f(x[end])/2
    
    for i in 2:n
        s += f(x[i])
    end
    
    return (b-a) * s/n
end
    

In [ ]:

Inn2 = trapezoidal.(a, b, f, nn);
err2 = abs.(Inn2 .- Ie);

In [ ]:
loglog(nn, err2, "bo")
xlabel("n")
ylabel("ε")
nn1 = 10:300
ee1 = 5 .* nn1.^(-2)
plot(nn1, ee1, "r--")
text(50, 0.005, L"$\mathcal{O}(n^{-2})$")

In [ ]:
@benchmark trapezoidal(a, b, f, 100000)

In [ ]:
trapezoidal(a, b, f, 100000) - Ie

In [ ]:
lmid = loglog(nn, err, "b-", label="Mid Point")
xlabel("n")
ylabel("ε")
ltrap = plot(nn, err2, "r-", label="Trapezoidal")
ltrap2 = plot(nn, 2err, "g:", label=L"2 $\times$ Mid Point")
legend()


# Regra de Simpson

Usa três pontos para aproximar uma parábola:

$$
\int_{i-1}^{i+1} f(x)\:dx = S_i(h) = \frac{h}{3}\left[f(x_{i-1}) + 4f(x_i) + f(x_{i+1})\right]
$$

Assim, no domínio inteiro, 

$$
\int_a^b f(x)\:dx \approx \frac{h}{3}\left[f(x_1) + 4f(x_2) + 2f(x_3) + 4f(x_4) + \ldots + 2f(x_{n-2}) + 4f(x_{n-1}) + f(x_n)\right]
$$

In [ ]:
# Detalhe n tem que ser divisível por 2.
function simpsonsrule(a, b, f, n)
    x = range(a, b, length=n+1)
    
    s = f(x[1]) + f(x[end])
    w, w2 = 4,2
    for i = 2:n
        s += w*f(x[i])
        w, w2 = w2, w
    end
    
    return s * (b-a)/(3n)
end

In [ ]:
nn2 = 2:2:1000
Inn3 = simpsonsrule.(a, b, f, nn2);
err3 = abs.(Inn3 .- Ie);

In [ ]:
loglog(nn2, err3, "bo")
xlabel("n")
ylabel("ε")
nn1 = 10:300
ee1 = 5 .* nn1.^(-4)
plot(nn1, ee1, "r--")
text(50, 0.000002, L"$\mathcal{O}(n^{-4})$")

In [ ]:
@benchmark simpsonsrule(a, b, f, 100000)

In [ ]:
simpsonsrule(a, b, f, 300) - Ie

In [ ]:
@benchmark simpsonsrule(a, b, f, 300)



# Generalização do processo

Usando as idéias acima com interpoladores polinomiais de ordem crescente pode-se obter regras de ordem superior para a integração. Com isso se chegam às formulas de Newton-Cotes (<https://en.wikipedia.org/wiki/Newton%E2%80%93Cotes_formulas>)

**Cuidado com isso!!!** Lembre-se das oscilações dos polinômios de Lagrange.


# Integração de Romberg

Um processo parecido com a extrapolação de Richardson (veremos isso mais tarde). Suponhamos que queremos estimar

$$
\int_a^b f(\xi) \: d\xi
$$

Uma primeira aproximação é usar a regra do trapézio nas extremidades:

$$
I = \int_a^b f(\xi) \: d\xi \approx T_0 = \frac{h_0}{2}\left[f(x_0) + f(x_0+h_0)\right] \qquad h_0 = b-a
$$

Dividindo os intervalos por dois sequencialmente, obtém-se:

$$
\begin{align*}
T_0 &= \frac{h_0}{2}\left[f(x_0) + f(x_0+h_0)\right] \\
T_1 &= \frac{h_1}{2}\left[f(x_0) + 2f(x_0+h_1) + f(x_0 + 2h_1)\right] \\
\vdots & \\
T_k &= \frac{h_k}{2}\left[f(x_0) + 2\sum_{i=1}^{2^k-1}f(x_0+ih_k) + f(x_0 + 2^kh_k)\right]
\end{align*}
$$
onde 

$$h_k = \frac{b-a}{2^k}$$

O erro da integral vale:

$$
\epsilon_k = I - T_k
$$

Lembrando que a regra do trapézio tem convergência quadrática,

$$
\epsilon_{k+1} \approx \frac{1}{4}\epsilon_k
$$

Assim, para $k=0, 1, \ldots$
$$
I = T_0 + \epsilon_0,\\
I = T_1 + \epsilon_1 \simeq  T_1 + \frac{1}{4}\epsilon_0 = T_1 + \frac{1}{4}(I - T_0) \Longrightarrow I \simeq \frac{4T_1 - T_0}{3}
$$

Este processo pode ser generalizado:

$$
I^1_k = \frac{4T_k - T_{k-1}}{3}
$$

In [ ]:
function romberg1(a, b, f, n)
    R = zeros(n, n)
    h = Float64(b-a)
    R[1,1] = h/2 * (f(a) - f(b))
    
    for i in 2:n
        s = 0.0
        for k in 1:(2^(i-2))
            s += f(a + (k-0.5)*h)
        end
        R[1,i] = 0.5 * (R[1,i-1] + h * s)
        for k in 2:i
            R[k,i] = R[k-1,i] + (R[k-1,i] - R[k-1,i-1]) / (4^(k-1) - 1)
        end
        h = h/2
    end
    
    return R
end


In [ ]:
romberg1(0, π, sin, 6)

In [ ]:
function romberg(a, b, f, n)
    R1 = zeros(n)
    R2 = zeros(n)
    h = Float64(b-a)
    R1[1] = h/2 * (f(a) - f(b))
    for i in 2:n
        s = 0.0
        for k in 1:(2^(i-2))
            s += f(a + (k-0.5)*h)
        end
        R2[1] = 0.5 * (R1[1] + h * s)
        for k in 2:i
            R2[k] = R2[k-1] + (R2[k-1] - R1[k-1]) / (4^(k-1) - 1)
        end
        for k in 1:i
            R1[k] = R2[k]
        end
        h = h/2
    end
    
    return R2[n]
    
end


In [ ]:
function romberg(a, b, f, n)
    R1 = zeros(n)
    R2 = zeros(n)
    h = Float64(b-a)
    R1[1] = h/2 * (f(a) - f(b))
    for i in 2:n
        s = 0.0
        for k in 1:(2^(i-2))
            s += f(a + (k-0.5)*h)
        end
        R2[1] = 0.5 * (R1[1] + h * s)
        for k in 2:i
            R2[k] = R2[k-1] + (R2[k-1] - R1[k-1]) / (4^(k-1) - 1)
        end
        for k in 1:i
            R1[k] = R2[k]
        end
        h = h/2
    end
    
    return R2[n]
    
end


In [ ]:
@btime romberg(a, b, f, 20)

In [ ]:

nn4 = 1:20
Inn4 = romberg.(a, b, f, nn4);
nh4 = 2 .^ nn3
err4 = abs.(Inn4 .- Ie);

In [ ]:
loglog(nn3, err4, "bo")

In [ ]:
f


## Abordagem usual

A abordagem mais comum é escolher uns nós e escolher pesos de integração adequados:

$$
\int_a^bf(\xi)\:d\xi \approx \sum_{i=1}^n w_i f(x_i)
$$

Falta agora determinar os pesos $w_i$. No caso de interpolação polynomial, usando os nós $x_i$,  *sempre* se pode encontrar interpoladores de Lagrange equivalentes:

$$
\sum_{i=1}^n a_i \phi_i(x) \equiv \sum_{i=1}^n f(x_i) h_i(x)
$$
onde $\phi_i(x)$ é parte de alguma família de *polinômios* e $h_i(x)$ é o interpolador de Lagrange


In [ ]:
?text

# Problemas

## Problema 1

Faça uma função para calcular a integral usando a regra do ponto médio com pontos cujo espaçamento não é igual


## Problema 2
Faça uma função para calcular a integral usando a regra do trapézio com pontos cujo espaçamento não é igual
